Here we present a quick example for computing matching diagrams and densities.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.spatial.distance as dist
import itertools

import tdqual.topological_data_quality_0 as tdqual

import os 
plots_dir = "plots/quickexample/"
os.makedirs(plots_dir, exist_ok=True)

Let us generate a point sample and visualise it.

In [ ]:
RandGen = np.random.default_rng(2)
# # Generate Random Sample
Z = tdqual.sampled_circle(0,2,30, RandGen)
X_indices = RandGen.choice(Z.shape[0], 15, replace=False)
Z[X_indices]+=[1,0]
X = Z[X_indices]
# Plot point cloud
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(Z[:,0], Z[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0.png")

Next, we sort the points so that the first $\# X$ points from $Z$ are those from $X$.

In [ ]:
# Sort indices of points in Z, so that 
X_compl = np.ones(Z.shape[0], dtype="bool")
X_compl[X_indices] = False
Z = np.vstack((Z[X_indices], Z[X_compl]))
X_indices = range(len(X_indices))
X = Z[X_indices]

Next, we compute the Minimum Spanning Trees $\textrm{MST}(X)$ and $\textrm{MST}(Z)$ for the pair $X\subseteq Z$.

In [ ]:
filtration_list_X, pairs_arr_X = tdqual.mst_edge_filtration(X) # MST(X)
filtration_list_Z, pairs_arr_Z = tdqual.mst_edge_filtration(Z) # MST(Z)

We can do the same computation by giving the distance matrices of $X$ and $Z$ respectively.

In [ ]:
import scipy.spatial.distance as dist
Dist_X = dist.squareform(dist.pdist(X))
Dist_Z = dist.squareform(dist.pdist(Z))
filtration_list_X_from_dist, pairs_arr_X_from_dist = tdqual.mst_edge_filtration(Dist_X, is_dist=True) # MST(X)
filtration_list_Z_from_dist, pairs_arr_Z_from_dist = tdqual.mst_edge_filtration(Dist_Z, is_dist=True) # MST(Z)
# Check that both outputs are equal
_tol=10e-8
assert len(filtration_list_X_from_dist)==len(filtration_list_X)
assert len(filtration_list_Z_from_dist)==len(filtration_list_Z)
assert np.all(np.abs(np.array(filtration_list_X_from_dist)-np.array(filtration_list_X))<_tol)
assert np.all(np.abs(np.array(filtration_list_Z_from_dist)-np.array(filtration_list_Z))<_tol)
assert pairs_arr_X_from_dist.shape==pairs_arr_X.shape
assert pairs_arr_Z_from_dist.shape==pairs_arr_Z.shape
assert np.all(np.abs(np.array(pairs_arr_X_from_dist)-np.array(pairs_arr_X))<_tol)
assert np.all(np.abs(np.array(pairs_arr_Z_from_dist)-np.array(pairs_arr_Z))<_tol)

Now, we compute the Triplet Merge Trees $\textrm{TMT}(X)$ and $\textrm{TMT}(Z)$

In [ ]:
TMT_X_pairs = tdqual.compute_tmt_pairs(filtration_list_X, pairs_arr_X)
TMT_Z_pairs = tdqual.compute_tmt_pairs(filtration_list_Z, pairs_arr_Z)

Next, we take from TMT_Z_pairs the pairs restricted to points from X

In [ ]:
indices_X_Z = np.max(TMT_Z_pairs, axis=1)<X.shape[0]
TMT_X_Z_pairs = TMT_Z_pairs[indices_X_Z]
indices_X_Z = np.nonzero(indices_X_Z)[0]

Next, we compute the associated matrix $F^X$, restricted to the indices of the subset, and the induced matching.

In [ ]:
FX = tdqual.get_inclusion_matrix(TMT_X_pairs, TMT_X_Z_pairs) # Associated matrix
matchingX = tdqual.get_inclusion_matrix_pivots(FX, Z.shape[0]) # Matching in TMT_X_Z
matching =[indices_X_Z[i] for i in matchingX] # Matching in all TMT_Z

Let us compute the matching barcodes.

In [ ]:
fig, ax = plt.subplots(figsize=(7,2.5))
tdqual.plot_matching_0(filtration_list_X, filtration_list_Z, matching, ax)
plt.tight_layout()
plt.savefig(plots_dir + "block_matching_0.png")

Next, we plot the matching diagram.

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
D_f, multiplicities = tdqual.compute_matching_diagram(filtration_list_X, filtration_list_Z, matching, _tol=1e-5)
tdqual.plot_matching_diagram(D_f, ax)
plt.tight_layout()
plt.savefig(plots_dir + "matching_diagram_0.png")

And finally, we plot the density of this matching diagram. For this, we compute first a representation of $D(f)$ and read also the points from the cokernel.

In [ ]:
# Representation of D(f)
D_f_rep = []
for i, pair in enumerate(D_f):
    for j in range(multiplicities[i]):
        D_f_rep += list(pair)

D_f_rep = np.array(D_f_rep).reshape(-1,2) # saved into a numpy array
# Representation of coker(f)
coker_f_X = D_f[D_f[:,0]==np.inf][:,1]
coker_f_mult = np.array(multiplicities)[D_f[:,0]==np.inf]
coker_f_rep = []
for i, end in enumerate(coker_f_X):
    coker_f_rep += [end]*coker_f_mult[i]

In [ ]:
tdqual.plot_density_matching_diagram(D_f_rep, coker_f_rep, plots_dir + "density_matrix_0.png", nbins=5, show_colorbar=True)